In [1]:
import pyspark

ModuleNotFoundError: No module named 'pyspark'

In [3]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.3.0-bin-hadoop2'

In [4]:
import pyspark

from pyspark.sql import SparkSession
  
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
  
df = spark.read.text("twitter_small.txt")
  
df.selectExpr("split(value, ' | ') as\
Text_Data_In_Rows_Using_Text").show(4,False)


In [5]:
from pyspark.sql import SparkSession 

     %% spark2= SparkSession.builder.getOrCreate()

In [6]:
import pyspark.sql.functions as f

 df2 = spark2.read.option("header", "False")\
    .option("delimiter", ", ")\
    .option("ignoreLeadingWhiteSpace", "False")\
    .option("ignoreTrailingWhiteSpace","False")\
    .option("inferScheme", "True") \
    .csv("twitter_small.txt")

df2.show()

In [ ]:
df2.printSchema()

print(df2.columns[0])
print(df2.columns[1])

Other solution.

In [7]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

schema = StructType([StructField('col1', LongType(), True),
                     StructField('col2', IntegerType(), True),
                     StructField('col3', IntegerType(), True)])
df_new = spark.read.csv("twitter_small.txt", schema=schema)
df_new.printSchema()

schema = StructType([StructField('col1', LongType(), True)])
spark.read.csv("twitter_small.txt",schema=schema).show()

### This is a json file!

In [ ]:
dfFromTxt=spark.read.text("twitter_small.txt")
dfFromTxt.printSchema()

In [ ]:

#Creating a scheme for the json file


from pyspark.sql.types import StructType,StructField, StringType
schema = StructType([ 
    StructField("created_at",StringType(),True), 
    StructField("id_str",StringType(),True), 
    StructField("text",StringType(),True), 
    StructField("user", StringType(), True)
  ])
from pyspark.sql.functions import col,from_json
dfJSON = dfFromTxt.withColumn("jsonData",from_json(col("value"),schema)) \
                   .select("jsonData.*")
dfJSON.printSchema()
dfJSON.show(truncate=False)

In [54]:
df23 = spark.read.json("twitter_small.json")
df23.printSchema()
df23.show()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

df23.select("contributors").show(truncate=False)

In [14]:
#importing tokenizer from pyspark

from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="full_text", outputCol="tokens")
tokens = tokenizer.transform(df23.select(col("full_text")))
tokens.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|full_text                                                                                                                                                                                                                                                                                           |tokens                                                                         

token_counts = tokens.select("full_text", f.explode("tokens").alias("token"))\
    .groupBy("full_text", "token").count()\
    .orderBy("full_text", "token")
token_counts.show(truncate=True, n=100)

tokens.select("full_text", f.explode("tokens").alias("token"))\
    .groupBy("full_text", "token")\
    .count()\
    .groupBy("full_text")\
    .agg(f.collect_list(f.struct(f.col("token"), f.col("count"))).alias("text"))\
    .orderBy("full_text")\
    .show(truncate=True)

In [17]:
#Latest solution
#Converted txt file to json
#reading all the words in full text and counting

df10 = spark.read.json("twitter_small.json")


df10.withColumn('word', f.explode(f.split(f.col("full_text"), ' '))) \
  .groupBy('word') \
  .count() \
  .sort('count', ascending=False) \
  .show()

+----+-----+
|word|count|
+----+-----+
|  de| 3114|
|  RT| 2236|
| het| 1845|
| een| 1841|
| van| 1803|
|  en| 1709|
|  in| 1613|
|  is| 1598|
| dat| 1319|
|  je| 1199|
|  op| 1020|
|  ik| 1015|
|niet|  995|
|  te|  993|
|voor|  924|
| met|  908|
|    |  850|
| die|  800|
|zijn|  631|
|  er|  623|
+----+-----+
only showing top 20 rows



In [55]:
#trying second tokenizer solution


tokenizer2 = Tokenizer(inputCol="full_text", outputCol="words_token")
tokenized = tokenizer2.transform(df10).select('full_text','words_token')

In [56]:
#seperates every word with "",""

print('############ Tokenized data extract:')
tokenized.show()

############ Tokenized data extract:
+--------------------+--------------------+
|           full_text|         words_token|
+--------------------+--------------------+
|RT @Desiree_Laver...|[rt, @desiree_lav...|
|@wovenwn @GSBdest...|[@wovenwn, @gsbde...|
|Barry Atsma per o...|[barry, atsma, pe...|
|RT @FAVV_Consumen...|[rt, @favv_consum...|
|@ANanninga @SilSc...|[@ananninga, @sil...|
|RT @ohboywhatasho...|[rt, @ohboywhatas...|
|@mariekehoogwout ...|[@mariekehoogwout...|
|RT @Moorkopje2: B...|[rt, @moorkopje2:...|
|RT @leolewin: Cen...|[rt, @leolewin:, ...|
|@st1ucia I CANT B...|[@st1ucia, ı, can...|
|Eyeshield https:/...|[eyeshield, https...|
|@SintSonja @Polit...|[@sintsonja, @pol...|
|RT @markdeholland...|[rt, @markdeholla...|
|Absoluut geen cou...|[absoluut, geen, ...|
|RT @seven__: Nede...|[rt, @seven__:, n...|
|RT @choi_bts2: Ji...|[rt, @choi_bts2:,...|
|RT @narkobars: Sh...|[rt, @narkobars:,...|
|@12_klisman wtf U...|[@12_klisman, wtf...|
|RT @VRH_Haaglande...|[rt, @vrh_haaglan

In [57]:
#including stop words to get cleaner results

#looked up online to see Dutch Stop words https://countwordsfree.com/stopwords/dutch

from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(stopWords=["aan","af","al","als","bij","dan","dat","die","dit","een","en","er","had","heb","hem","het","hij","hoe","hun","ik","in","is","je","kan","me","men","met","mij","nog","nu","of","ons","ook","te","tot","uit","van","was","wat","we","wel","wij","zal","ze","zei","zij","zo","zou","aangaande","aangezien","achter","achterna","afgelopen","aldaar","aldus","alhoewel","alias","alle","allebei","alleen","alsnog","altijd","altoos","ander","andere","anders","anderszins","behalve","behoudens","beide","beiden","ben","beneden","bent","bepaald","betreffende","binnen","binnenin","boven","bovenal","bovendien","bovengenoemd","bovenstaand","bovenvermeld","buiten","daar","daarheen","daarin","daarna","daarnet","daarom","daarop","daarvanlangs","de","dikwijls","door","doorgaand","dus","echter","eer","eerdat","eerder","eerlang","eerst","elk","elke","enig","enigszins","enkel","erdoor","even","eveneens","evenwel","gauw","gedurende","geen","gehad","gekund","geleden","gelijk","gemoeten","gemogen","geweest","gewoon","gewoonweg","haar","hadden","hare","hebben","hebt","heeft","hen","hierbeneden","hierboven","hoewel","hunne","ikzelf","inmiddels","inzake","jezelf","jij","jijzelf","jou","jouw","jouwe","juist","jullie","klaar","kon","konden","krachtens","kunnen","kunt","later","liever","maar","mag","meer","mezelf","mijn","mijnent","mijner","mijzelf","misschien","mocht","mochten","moest","moesten","moet","moeten","mogen","na","naar","nadat","net","niet","noch","nogal","ofschoon","om","omdat","omhoog","omlaag","omstreeks","omtrent","omver","onder","ondertussen","ongeveer","onszelf","onze","op","opnieuw","opzij","over","overeind","overigens","pas","precies","reeds","rond","rondom","sedert","sinds","sindsdien","slechts","sommige","spoedig","steeds","tamelijk","tenzij","terwijl","thans","tijdens","toch","toen","toenmaals","toenmalig","totdat","tussen","uitgezonderd","vaakwat","vandaan","vanuit","vanwege","veeleer","verder","vervolgens","vol","volgens","voor","vooraf","vooral","vooralsnog","voorbij","voordat","voordezen","voordien","voorheen","voorop","vooruit","vrij","vroeg","waar","waarom","wanneer","want","waren","weer","weg","wegens","weldra","welk","welke","wie","wiens","wier","wijzelf","zelfs","zichzelf","zijn","zijne","zodra","zonder","zouden","zowat","zulke","zullen","zult","rt","ık","deze","u","via"], inputCol='words_token', outputCol='words_clean')
data_clean = remover.transform(tokenized).select('words_token', 'words_clean')

print('############ Data Cleaning extract:')
data_clean.show()

############ Data Cleaning extract:
+--------------------+--------------------+
|         words_token|         words_clean|
+--------------------+--------------------+
|[rt, @desiree_lav...|[@desiree_laverne...|
|[@wovenwn, @gsbde...|[@wovenwn, @gsbde...|
|[barry, atsma, pe...|[barry, atsma, pe...|
|[rt, @favv_consum...|[@favv_consument:...|
|[@ananninga, @sil...|[@ananninga, @sil...|
|[rt, @ohboywhatas...|[@ohboywhatashot:...|
|[@mariekehoogwout...|[@mariekehoogwout...|
|[rt, @moorkopje2:...|[@moorkopje2:, bi...|
|[rt, @leolewin:, ...|[@leolewin:, cens...|
|[@st1ucia, ı, can...|[@st1ucia, ı, can...|
|[eyeshield, https...|[eyeshield, https...|
|[@sintsonja, @pol...|[@sintsonja, @pol...|
|[rt, @markdeholla...|[@markdehollander...|
|[absoluut, geen, ...|[absoluut, coulan...|
|[rt, @seven__:, n...|[@seven__:, neder...|
|[rt, @choi_bts2:,...|[@choi_bts2:, jin...|
|[rt, @narkobars:,...|[@narkobars:, sho...|
|[@12_klisman, wtf...|[@12_klisman, wtf...|
|[rt, @vrh_haaglan...|[@vrh_haaglanden:.

In [59]:
#when I list after filtering out 200 ish words 

result = data_clean.withColumn('word', f.explode(f.col('words_clean'))) \
  .groupBy('word') \
  .count().sort('count', ascending=False) \

print('############')
result.show(200)


############
+----------------+-----+
|            word|count|
+----------------+-----+
|                | 1982|
|               -|  441|
|            gaat|  241|
|          mensen|  226|
|            goed|  209|
|           wordt|  203|
|            echt|  192|
|            veel|  177|
|               1|  169|
|            eens|  163|
|            hier|  157|
|            gaan|  150|
|             wil|  143|
|              ın|  141|
|          worden|  139|
|            heel|  138|
|             omg|  131|
|          nieuwe|  131|
|              ja|  124|
|           tegen|  122|
|            iets|  121|
|            jaar|  120|
|       #vacature|  118|
|            @nos|  112|
|            weet|  109|
|            komt|  105|
|               ||  104|
|              ga|  100|
|              ıs|   98|
|             mee|   97|
|             dag|   97|
|            denk|   95|
|            doen|   94|
|             man|   93|
|            rıvm|   91|
|           alles|   87|
|           

#stopwords
"aan","af","al","als","bij","dan","dat","die","dit","een","en","er","had","heb","hem","het","hij","hoe","hun","ik","in","is","je","kan","me","men","met","mij","nog","nu","of","ons","ook","te","tot","uit","van","was","wat","we","wel","wij","zal","ze","zei","zij","zo","zou","aangaande","aangezien","achter","achterna","afgelopen","aldaar","aldus","alhoewel","alias","alle","allebei","alleen","alsnog","altijd","altoos","ander","andere","anders","anderszins","behalve","behoudens","beide","beiden","ben","beneden","bent","bepaald","betreffende","binnen","binnenin","boven","bovenal","bovendien","bovengenoemd","bovenstaand","bovenvermeld","buiten","daar","daarheen","daarin","daarna","daarnet","daarom","daarop","daarvanlangs","de","dikwijls","door","doorgaand","dus","echter","eer","eerdat","eerder","eerlang","eerst","elk","elke","enig","enigszins","enkel","erdoor","even","eveneens","evenwel","gauw","gedurende","geen","gehad","gekund","geleden","gelijk","gemoeten","gemogen","geweest","gewoon","gewoonweg","haar","hadden","hare","hebben","hebt","heeft","hen","hierbeneden","hierboven","hoewel","hunne","ikzelf","inmiddels","inzake","jezelf","jij","jijzelf","jou","jouw","jouwe","juist","jullie","klaar","kon","konden","krachtens","kunnen","kunt","later","liever","maar","mag","meer","mezelf","mijn","mijnent","mijner","mijzelf","misschien","mocht","mochten","moest","moesten","moet","moeten","mogen","na","naar","nadat","net","niet","noch","nogal","ofschoon","om","omdat","omhoog","omlaag","omstreeks","omtrent","omver","onder","ondertussen","ongeveer","onszelf","onze","op","opnieuw","opzij","over","overeind","overigens","pas","precies","reeds","rond","rondom","sedert","sinds","sindsdien","slechts","sommige","spoedig","steeds","tamelijk","tenzij","terwijl","thans","tijdens","toch","toen","toenmaals","toenmalig","totdat","tussen","uitgezonderd","vaakwat","vandaan","vanuit","vanwege","veeleer","verder","vervolgens","vol","volgens","voor","vooraf","vooral","vooralsnog","voorbij","voordat","voordezen","voordien","voorheen","voorop","vooruit","vrij","vroeg","waar","waarom","wanneer","want","waren","weer","weg","wegens","weldra","welk","welke","wie","wiens","wier","wijzelf","zelfs","zichzelf","zijn","zijne","zodra","zonder","zouden","zowat","zulke","zullen","zult"